In [1]:
load('quandles-alpha.sage')

In [2]:
# import (in a plain sage session you usually already have these)
from sage.all import Knot, Knots, BraidGroup

# 1. from the Rolfsen table (3_1 is the trefoil)
K = Knots().from_table(3,1)    # K is now the trefoil knot
print(K)                       # e.g. "Knot represented by 3 crossings"

# 2. from a braid (another way to get the trefoil)
B = BraidGroup(2)
trefoil = Knot(B([1,1,1]))     # braid word [1,1,1] gives the trefoil
print(trefoil)

# 3. from a Dowker/DT code or Gauss code
K2 = Knots().from_dowker_code([8,10,2,12,4,6])
K3 = Knots().from_gauss_code([2, -1, 3, -2, 1, -3])

# in the case of Gauss code, the sign shows the knot's over or undercrossing
#specifically, we can label each segment following the labels in the gauss code, that is if the first crossing is called 2, then the 
# segment overcrossing is called 2, note that we can not multiply the whole expression by -1 (produce the mirror image) but we can 
#shifting the expression so that it starts with 1. for example, the above notation can be changed into 
#K3 = Knots().from_gauss_code([1, -3, 2, -1, 3, -2])
# from this expression, we can know seg 1 goes through crossing 1 and become segment 3 after underpassing crossing 3, and then
# overpassing 2 ... so it is uniquely determined. In this case, we can uniquely define...

K8 = [-1, 4, -2, 3, 1, -4, 2, -3] #figure-8 knot

Knot represented by 3 crossings
Knot represented by 3 crossings


In [3]:
# Implementation of your updated algorithm:
# - Build arcs: for each negative gauss entry at index i, find the next negative entry at j and form arc [c, gauss[i+1], ..., gauss[j-1]]
# - Remove arcs contained inside earlier arcs
# - For each crossing a:
#     x = arc that includes an occurrence of a (in any position inside the arc)
#     find index of -a in gauss -> y = arc that contains that index
#     b = element before -a -> z = arc that contains prev index
# - Sign: parity rule on endpoints between a and -a (even -> +1, odd -> -1)

from typing import List, Dict, Any

def in_interval_mod(idx: int, start: int, end: int, n: int) -> bool:
    if start <= end:
        return start <= idx < end
    else:
        return idx >= start or idx < end

def build_arcs_neg_to_next_neg(gauss: List[int]) -> List[Dict[str, Any]]:
    """
    For each negative entry gauss[i] (value c < 0), find next negative at index j>i (cyclic).
    Form arc = [c, gauss[i+1], ..., gauss[j-1]]  (note: does NOT include gauss[j]).
    Return list of arcs with metadata, then remove arcs fully contained in previous arcs.
    """
    n = len(gauss)
    gauss = [int(x) for x in gauss]
    raw_arcs = []
    for i, val in enumerate(gauss):
        if val < 0:
            # find next negative
            j = (i + 1) % n
            found = False
            while True:
                if gauss[j] < 0:
                    found = True
                    break
                j = (j + 1) % n
                if j == i:
                    break
            if not found:
                continue
            # indices i .. j-1 cyclic
            idxs = []
            k = i
            while True:
                idxs.append(k % n)
                k = (k + 1) % n
                if (k % n) == j:
                    break
            vals = [gauss[k] for k in idxs]
            raw_arcs.append({
                'start_idx': i,
                'end_idx': (j-1) % n,
                'indices': idxs,
                'values': vals,
                'label': abs(val)   # label by absolute of starting negative
            })
    # remove arcs that are contained entirely in an earlier kept arc
    kept = []
    for arc in raw_arcs:
        s = set(arc['indices'])
        is_subset = False
        for k in kept:
            if s.issubset(set(k['indices'])):
                is_subset = True
                break
        if not is_subset:
            kept.append(arc)
    return kept

def crossing_positions(gauss: List[int]) -> Dict[int, List[int]]:
    gauss = [int(x) for x in gauss]
    pos = {}
    for i, v in enumerate(gauss):
        k = abs(v)
        pos.setdefault(k, []).append(i)
    return pos

def parity_signs_from_gauss(gauss: List[int]) -> Dict[int,int]:
    """
    Standard parity rule: for crossing a (positions p,q), count number of other crossings
    with exactly one endpoint in [p,q) (cyclic). Even -> +1, odd -> -1.
    """
    n = len(gauss)
    gauss = [int(x) for x in gauss]
    posmap = crossing_positions(gauss)
    signs = {}
    labels = sorted(posmap.keys())
    for a in labels:
        p, q = posmap[a]
        count = 0
        for b in labels:
            if b == a: continue
            p1, p2 = posmap[b]
            inside1 = in_interval_mod(p1, p, q, n)
            inside2 = in_interval_mod(p2, p, q, n)
            if inside1 ^ inside2:
                count += 1
        signs[a] = 1 if (count % 2 == 0) else -1
    return signs

def analyze_gauss_updated(gauss: List[int]) -> Dict[str,Any]:
    gauss = [int(x) for x in gauss]
    n = len(gauss)
    if n % 2 != 0:
        raise ValueError("Gauss length must be even.")
    # 1) build arcs per updated rule
    arcs = build_arcs_neg_to_next_neg(gauss)
    # map traversal index -> first arc id that contains it
    index_to_arc = {}
    for aid, arc in enumerate(arcs, start=1):
        for idx in arc['indices']:
            if idx not in index_to_arc:
                index_to_arc[idx] = aid
    # 2) crossing positions and parity signs
    posmap = crossing_positions(gauss)
    signs = parity_signs_from_gauss(gauss)
    # 3) for each crossing compute x,y,z per your text
    crossing_info = {}
    for a in sorted(posmap.keys()):
        p, q = posmap[a]
        # x: the arc that includes an occurrence of 'a' (value +a or -a anywhere inside arc's values)
        x_arc = None
        for aid, arc in enumerate(arcs, start=1):
            if any(abs(v) == a for v in arc['values']):
                x_arc = aid
                break
        # find index of -a (the over-visit; if not found, use fallback)
        try:
            over_idx = next(i for i,v in enumerate(gauss) if v == -a)
        except StopIteration:
            over_idx = None
        y_arc = index_to_arc.get(over_idx) if over_idx is not None else None
        # b = element behind -a
        if over_idx is None:
            prev_idx = None
            b_val = None
        else:
            prev_idx = (over_idx - 1) % n
            b_val = gauss[prev_idx]
        z_arc = index_to_arc.get(prev_idx) if prev_idx is not None else None
        crossing_info[a] = {
            'positions': posmap[a],
            'x_arc': x_arc,
            'y_arc': y_arc,
            'z_arc': z_arc,
            'b_value': b_val,
            'sign_parity': signs[a]
        }
    return {
        'gauss': gauss,
        'arcs': arcs,
        'index_to_arc': index_to_arc,
        'positions': posmap,
        'crossing_info': crossing_info,
        'signs': signs
    }

# ---------- Example runs (trefoil and figure-eight) ----------
if __name__ == "__main__":
    examples = {
        'trefoil': [-1, 3, -2, 1, -3, 2],
        'figure8': [-1, 4, -2, 3, 1, -4, 2, -3]
    }
    for name, g in examples.items():
        print("\n---", name, "Gauss:", g)
        out = analyze_gauss_updated(g)
        print("\nArcs (id : label, indices, values):")
        for i, arc in enumerate(out['arcs'], start=1):
            print(f" arc {i}: label={arc['label']}, indices={arc['indices']}, values={arc['values']}")
        print("\nIndex -> arc (first containing index):", out['index_to_arc'])
        print("\nCrossing info (label: x_arc, y_arc, z_arc, b_value, sign):")
        for lab in sorted(out['crossing_info'].keys()):
            info = out['crossing_info'][lab]
            print(f" {lab}: x={info['x_arc']}, y={info['y_arc']}, z={info['z_arc']}, b={info['b_value']}, sign={info['sign_parity']}")



--- trefoil Gauss: [-1, 3, -2, 1, -3, 2]

Arcs (id : label, indices, values):
 arc 1: label=1, indices=[0, 1], values=[-1, 3]
 arc 2: label=2, indices=[2, 3], values=[-2, 1]
 arc 3: label=3, indices=[4, 5], values=[-3, 2]

Index -> arc (first containing index): {0: 1, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3}

Crossing info (label: x_arc, y_arc, z_arc, b_value, sign):
 1: x=1, y=1, z=3, b=2, sign=1
 2: x=2, y=2, z=1, b=3, sign=1
 3: x=1, y=3, z=2, b=1, sign=-1

--- figure8 Gauss: [-1, 4, -2, 3, 1, -4, 2, -3]

Arcs (id : label, indices, values):
 arc 1: label=1, indices=[0, 1], values=[-1, 4]
 arc 2: label=2, indices=[2, 3, 4], values=[-2, 3, 1]
 arc 3: label=4, indices=[5, 6], values=[-4, 2]
 arc 4: label=3, indices=[7], values=[-3]

Index -> arc (first containing index): {0: 1, 1: 1, 2: 2, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4}

Crossing info (label: x_arc, y_arc, z_arc, b_value, sign):
 1: x=1, y=1, z=4, b=-3, sign=-1
 2: x=2, y=2, z=1, b=4, sign=-1
 3: x=2, y=4, z=3, b=2, sign=1
 4: x=1, y=3, z=2, b=1,

In [5]:
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

# ---------------- utility functions (unchanged logic) ----------------
def in_interval_mod(idx: int, start: int, end: int, n: int) -> bool:
    if start <= end:
        return start <= idx < end
    else:
        return idx >= start or idx < end

def build_arcs_neg_to_next_neg(gauss: List[int]) -> List[Dict[str, Any]]:
    n = len(gauss)
    gauss = [int(x) for x in gauss]
    raw_arcs = []
    for i, val in enumerate(gauss):
        if val < 0:
            j = (i + 1) % n
            found = False
            while True:
                if gauss[j] < 0:
                    found = True
                    break
                j = (j + 1) % n
                if j == i:
                    break
            if not found:
                continue
            idxs = []
            k = i
            while True:
                idxs.append(k % n)
                k = (k + 1) % n
                if (k % n) == j:
                    break
            vals = [gauss[k] for k in idxs]
            raw_arcs.append({
                'start_idx': i,
                'end_idx': (j-1) % n,
                'indices': idxs,
                'values': vals,
                'label': abs(val)
            })
    # remove arcs that are contained entirely in an earlier kept arc
    kept = []
    for arc in raw_arcs:
        s = set(arc['indices'])
        is_subset = False
        for k in kept:
            if s.issubset(set(k['indices'])):
                is_subset = True
                break
        if not is_subset:
            kept.append(arc)
    return kept

def crossing_positions(gauss: List[int]) -> Dict[int, List[int]]:
    gauss = [int(x) for x in gauss]
    pos = {}
    for i, v in enumerate(gauss):
        k = abs(v)
        pos.setdefault(k, []).append(i)
    return pos

def parity_signs_from_gauss(gauss: List[int]) -> Dict[int,int]:
    n = len(gauss)
    gauss = [int(x) for x in gauss]
    posmap = crossing_positions(gauss)
    signs = {}
    labels = sorted(posmap.keys())
    for a in labels:
        p, q = posmap[a]
        count = 0
        for b in labels:
            if b == a: continue
            p1, p2 = posmap[b]
            inside1 = in_interval_mod(p1, p, q, n)
            inside2 = in_interval_mod(p2, p, q, n)
            if inside1 ^ inside2:
                count += 1
        signs[a] = 1 if (count % 2 == 0) else -1
    return signs

# ---------------- dataclass to hold the result ----------------
@dataclass
class KnotInfo:
    gauss: List[int]
    arcs: List[Dict[str, Any]]                 # list of arc dicts: {start_idx,end_idx,indices,values,label}
    index_to_arc: Dict[int, int]               # traversal index -> first arc id containing it
    positions: Dict[int, List[int]]            # crossing label -> two visit indices
    crossings: Dict[int, Dict[str, Any]]       # crossing label -> info dict (x_arc,y_arc,z_arc,b_value,sign_parity,...)
    signs: Dict[int,int]                       # crossing label -> +1/-1

# ---------------- main analyzer (returns KnotInfo) ----------------
def analyze_gauss_updated(gauss: List[int]) -> KnotInfo:
    gauss = [int(x) for x in gauss]
    n = len(gauss)
    if n % 2 != 0:
        raise ValueError("Gauss length must be even.")
    arcs = build_arcs_neg_to_next_neg(gauss)
    index_to_arc: Dict[int, int] = {}
    for aid, arc in enumerate(arcs, start=1):
        for idx in arc['indices']:
            if idx not in index_to_arc:
                index_to_arc[idx] = aid
    posmap = crossing_positions(gauss)
    signs = parity_signs_from_gauss(gauss)
    crossing_info: Dict[int, Dict[str, Any]] = {}
    for a in sorted(posmap.keys()):
        p, q = posmap[a]
        x_arc: Optional[int] = None
        for aid, arc in enumerate(arcs, start=1):
            if any(abs(v) == a for v in arc['values']):
                x_arc = aid
                break
        try:
            over_idx = next(i for i,v in enumerate(gauss) if v == -a)
        except StopIteration:
            over_idx = None
        y_arc = index_to_arc.get(over_idx) if over_idx is not None else None
        if over_idx is None:
            prev_idx = None
            b_val = None
        else:
            prev_idx = (over_idx - 1) % n
            b_val = gauss[prev_idx]
        z_arc = index_to_arc.get(prev_idx) if prev_idx is not None else None
        crossing_info[a] = {
            'positions': posmap[a],
            'x_arc': x_arc,
            'y_arc': y_arc,
            'z_arc': z_arc,
            'b_value': b_val,
            'sign_parity': signs[a]
        }
    return KnotInfo(
        gauss=gauss,
        arcs=arcs,
        index_to_arc=index_to_arc,
        positions=posmap,
        crossings=crossing_info,
        signs=signs
    )

# ---------------- calculate_longitude(knot_info) ----------------
def calculate_longitude(knot_info: KnotInfo) -> Dict[str,Any]:
    """
    Build an UNREDUCED longitude from knot_info produced by analyze_gauss_updated.
    Returns:
      {
        'pairs': [(gen_index, exponent), ...],   # gen_index corresponds to arc id (1..m)
        'word_str': readable string like "g1 * g3^-1 * g2",
        'generators': {arc_id: 'g_k', ...}
      }
    """
    gauss = knot_info.gauss
    n = len(gauss)
    arcs = knot_info.arcs
    m = len(arcs)
    # generator names g1..gm
    gen_map = {i+1: f"g{i+1}" for i in range(m)}
    index_to_arc = knot_info.index_to_arc
    signs = knot_info.signs
    # determine visit roles: which index is under and which is over for each crossing
    visit_role: Dict[int, Tuple[int,int]] = {}
    posmap = knot_info.positions
    for label, (p,q) in posmap.items():
        # assume gauss neg = over, pos = under (if your convention differs, adjust)
        if gauss[p] < 0 and gauss[q] > 0:
            over_pos, under_pos = p, q
        elif gauss[q] < 0 and gauss[p] > 0:
            over_pos, under_pos = q, p
        else:
            # fallback: choose p as under, q as over (user may adjust if needed)
            under_pos, over_pos = p, q
        visit_role[label] = (under_pos, over_pos)
    # traverse in order and append over-generator^sign whenever encountering an under-visit
    longitude_pairs: List[Tuple[int,int]] = []
    for i, v in enumerate(gauss):
        lbl = abs(v)
        under_pos, over_pos = visit_role[lbl]
        if i == under_pos:
            gen_idx = index_to_arc.get(over_pos)
            if gen_idx is None:
                # skip if we cannot find arc containing the over_pos
                continue
            exp = signs.get(lbl, 1)
            longitude_pairs.append((gen_idx, exp))
    # make readable string
    def pair_to_str(pair):
        g, e = pair
        name = gen_map[g]
        return f"{name}^{e}" if e != 1 else f"{name}"
    word_str = " * ".join(pair_to_str(p) for p in longitude_pairs) if longitude_pairs else "1"
    return {
        'pairs': longitude_pairs,
        'word_str': word_str,
        'generators': gen_map
    }

# ---------------- example usage ----------------
if __name__ == "__main__":
    trefoil = [-1, 3, -2, 1, -3, 2]
    fig8 = [-1, 4, -2, 3, 1, -4, 2, -3]

    for name, g in [('trefoil', trefoil), ('figure8', fig8)]:
        print("\n---", name, "Gauss:", g)
        knot_info = analyze_gauss_updated(g)
        print("knot_info.arcs:")
        for i, arc in enumerate(knot_info.arcs, start=1):
            print(f" arc {i}: label={arc['label']}, indices={arc['indices']}, values={arc['values']}")
        print("\nknot_info.crossings:")
        for lab, info in knot_info.crossings.items():
            print(f" {lab}: {info}")
        long_info = calculate_longitude(knot_info)
        print("\nLongitude (unreduced):", long_info['word_str'])



--- trefoil Gauss: [-1, 3, -2, 1, -3, 2]
knot_info.arcs:
 arc 1: label=1, indices=[0, 1], values=[-1, 3]
 arc 2: label=2, indices=[2, 3], values=[-2, 1]
 arc 3: label=3, indices=[4, 5], values=[-3, 2]

knot_info.crossings:
 1: {'positions': [0, 3], 'x_arc': 1, 'y_arc': 1, 'z_arc': 3, 'b_value': 2, 'sign_parity': 1}
 2: {'positions': [2, 5], 'x_arc': 2, 'y_arc': 2, 'z_arc': 1, 'b_value': 3, 'sign_parity': 1}
 3: {'positions': [1, 4], 'x_arc': 1, 'y_arc': 3, 'z_arc': 2, 'b_value': 1, 'sign_parity': -1}

Longitude (unreduced): g3^-1 * g1 * g2

--- figure8 Gauss: [-1, 4, -2, 3, 1, -4, 2, -3]
knot_info.arcs:
 arc 1: label=1, indices=[0, 1], values=[-1, 4]
 arc 2: label=2, indices=[2, 3, 4], values=[-2, 3, 1]
 arc 3: label=4, indices=[5, 6], values=[-4, 2]
 arc 4: label=3, indices=[7], values=[-3]

knot_info.crossings:
 1: {'positions': [0, 4], 'x_arc': 1, 'y_arc': 1, 'z_arc': 4, 'b_value': -3, 'sign_parity': -1}
 2: {'positions': [2, 6], 'x_arc': 2, 'y_arc': 2, 'z_arc': 1, 'b_value': 4, 's

In [7]:
g = trefoil = [-1, 3, -2, 1, -3, 2]
knot_info = analyze_gauss_updated(g)
print(knot_info)

KnotInfo(gauss=[-1, 3, -2, 1, -3, 2], arcs=[{'start_idx': 0, 'end_idx': 1, 'indices': [0, 1], 'values': [-1, 3], 'label': 1}, {'start_idx': 2, 'end_idx': 3, 'indices': [2, 3], 'values': [-2, 1], 'label': 2}, {'start_idx': 4, 'end_idx': 5, 'indices': [4, 5], 'values': [-3, 2], 'label': 3}], index_to_arc={0: 1, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3}, positions={1: [0, 3], 3: [1, 4], 2: [2, 5]}, crossings={1: {'positions': [0, 3], 'x_arc': 1, 'y_arc': 1, 'z_arc': 3, 'b_value': 2, 'sign_parity': 1}, 2: {'positions': [2, 5], 'x_arc': 2, 'y_arc': 2, 'z_arc': 1, 'b_value': 3, 'sign_parity': 1}, 3: {'positions': [1, 4], 'x_arc': 1, 'y_arc': 3, 'z_arc': 2, 'b_value': 1, 'sign_parity': -1}}, signs={1: 1, 2: 1, 3: -1})


In [ ]:
print('alexander_polynomial =', K.alexander_polynomial())  # Alexander polynomial (Laurent polynomial in t)
print('jones_polynomial =', K.jones_polynomial())          # Jones polynomial
print('homfly_polynomial =', K.homfly_polynomial())        # HOMFLY polynomial
print('determinant =', K.determinant())                    # determinant
print('signature =', K.signature())                        # signature
print('genus =', K.genus())                                # genus of the knot
print('arf_invariant =', K.arf_invariant())                # Arf invariant
print('dt_code =', K.dt_code())                            # Dowker-Thistlethwaite code
print('braid =', K.braid())                                # braid representation (if available)          # prints a braid representation (if available)

In [ ]:
def positive_crossing(x,y,z):
    condition2 = g_z == g_y.quandleUnder(g_x)
    return condition1
def negative_crossing(n):
    condition2 = g_y == g_z.quandleUnder(g_x)
    return condition2   
def longitude(link_component):
    for (x,y,z) in link_component: